<a href="https://colab.research.google.com/github/samiravaez/medical-text-data-classifier/blob/main/MedicalTextDeepClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('drive')
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import gensim
import pandas as pd
import os
import glob
import numpy as np
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
from sklearn import preprocessing



target_names = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']

def get_files_in_dir(directory):
    file_names_list = []
    for file_name in glob.glob(directory):
        file_names_list += [file_name]
    return file_names_list


def get_text_file_data(file):
    data = []
    with open(file, 'r', encoding="latin-1") as file:
      for line in file:
        data += [str(line)]
    return data


content = []
labels = []

for i in range(1,24): #this should be 24
  p = 'C'+'%.2d' % i
  file_path = os.path.join('drive/MyDrive/ohsumed-all',str(p),'*')
  for file in get_files_in_dir(file_path):
     file_data = get_text_file_data(file)
     content += ["".join(file_data)]
     labels += [i]



# word2vec vectorization
features_data = pd.DataFrame(data=content, columns=['medText'])
new_data = features_data.medText.apply(gensim.utils.simple_preprocess)
# word2vec_model = gensim.models.Word2Vec(sentences=new_data, vector_size=100, window=5, sg=1)
# word2vec_model.save("word2vec.model")

#### loading the word2vec model
word2vec_model = gensim.models.Word2Vec.load('drive/MyDrive/word2vec.model')
word_vectors = word2vec_model.wv.vectors

# data pre-process
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(content)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(content)

# Padding sequences to have a consistent length
data = pad_sequences(sequences, maxlen=max_sequence_length)


#split data
labels = np.array(labels)
features_train, features_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.35, random_state=10)

# defining one-hot encoding for multi-class scenario
labels_onehot = to_categorical(labels_train, num_classes=len(target_names))
labels_test_onehot = to_categorical(labels_test, num_classes=len(target_names))


# Define the CNN model
keras_model = Sequential()
keras_model.add(Embedding(
    input_dim=word_vectors.shape[0],
    output_dim=word_vectors.shape[1],
    input_length=max_sequence_length,
    weights=[word_vectors],
    trainable=False
))
keras_model.add(Conv1D(128, 5, activation='relu', padding='same'))
keras_model.add(Conv1D(128, 5, activation='relu', padding='same'))
keras_model.add(MaxPool1D())
keras_model.add(Conv1D(200, 5, activation='relu', padding='same'))
keras_model.add(Conv1D(200, 5, activation='relu', padding='same'))
keras_model.add(GlobalMaxPool1D())
keras_model.add(Dense(64, activation='relu'))
keras_model.add(Dropout(0.2))
keras_model.add(Dense(len(target_names), activation='softmax'))


keras_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
keras_model.fit(features_train, labels_onehot, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model on the test data
loss, accuracy = keras_model.evaluate(features_test, labels_test_onehot)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
